In [1]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sqlite3
import os

In [2]:
# ワールドから2つの座標系を作成
# hash1 = "58145dd0e110aee0f8323bafe84bfdcf4f5568093503b0f7b9cf8dab822b9a12"
hash1 = "323"
# hash2 = "e8ecf3917e93f079f2b724871fee3aee314f445a0f824e038479f0e21b58c7e6"
hash2 = "525"
# hash3 = "ad86a80651f52dea498c34a2d3c766d327d8ed7a1d8900fac92b32ddfc859b8a"
hash3 = "433"

In [3]:
def generateAffineMatrix3D(matrix_affineA, matrix_affineAdash):
    # 各行列から平行移動を含む点を抽出
    points_A = np.array([mat[:3, 3] for mat in matrix_affineA])  # Aの各点 (x, y, z)
    points_Adash = np.array([mat[:3, 3] for mat in matrix_affineAdash])  # Adashの各点 (x', y', z')
    
    # アフィン変換行列を求めるための最小二乗法
    ones = np.ones((points_A.shape[0], 1))  # 点数分の1列
    P = np.hstack((points_A, ones))  # Aの点を拡張 [x, y, z, 1]
    # print(P)
    Q = points_Adash  # Adashの点
    # print(Q)
    
    # 最小二乗法で解を求める
    affine_matrix, _, _, _ = np.linalg.lstsq(P, Q, rcond=None)
    print("affine_matrix")
    print(affine_matrix)

    # 4x4のアフィン変換行列を構築
    affine_matrix_4x4 = np.eye(4)
    affine_matrix_4x4[:3, :] = affine_matrix.T
    
    return affine_matrix_4x4

def generateAffineMatrix3DSelf(matrix_affineA, matrix_affineAdash):
    # Aの点を拡張 [x, y, z, 1]
    points_A = np.array([mat[:3, 3] for mat in matrix_affineA])
    points_Adash = np.array([mat[:3, 3] for mat in matrix_affineAdash]) 
    
    # P = points_A
    P = np.hstack((points_A, np.ones((points_A.shape[0], 1))))
    # P = np.vstack((P, np.array([1, 1, 1, 1])))
    # print(P)
    Q = points_Adash
    # Q = np.hstack((points_Adash, np.ones((points_Adash.shape[0], 1))))
    # Q = np.vstack((Q, np.array([1, 1, 1, 1])))

    # print(Q)
    
    # 最小二乗法で解を求める
    affine_matrix = leastSquaresMethod(P, Q)
    
    # 4x4のアフィン変換行列を構築
    affine_matrix = np.vstack((affine_matrix.T, np.array([0, 0, 0, 1])))
    
    return affine_matrix

def generateAffineMatrix3DSelfLU(matrix_affineA, matrix_affineAdash):
    """LU分解を用いた最小二乗法でアフィン変換行列を求める"""
    points_A = np.array([mat[:3, 3] for mat in matrix_affineA])
    points_Adash = np.array([mat[:3, 3] for mat in matrix_affineAdash]) 
    
    # 拡張行列を作成
    P = np.hstack((points_A, np.ones((points_A.shape[0], 1))))
    # print(P)
    Q = points_Adash
    # print(Q)

    # 最小二乗法で解を求める
    affine_matrix = leastSquaresMethodLU(P, Q)

    # 4x4のアフィン変換行列を構築
    affine_matrix = np.vstack((affine_matrix.T, np.array([0, 0, 0, 1])))

    return affine_matrix

def LU(A: np.array) -> np.array:
    """LU分解"""
    n = A.shape[0]
    L = np.eye(n)
    U = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            U[i, j] = A[i, j] - np.sum(L[i, :i] * U[:i, j])
        for j in range(i + 1, n):
            L[j, i] = (A[j, i] - np.sum(L[j, :i] * U[:i, i])) / U[i, i]

    print(L)
    print(U)
    return L, U

def eq_solve(P: np.array, Q: np.array) -> np.array:
    # LU分解
    L, U = LU(P)
    n, m = Q.shape  # Q の形状を取得
    y = np.zeros((n, m))  # 前進代入の解ベクトル y を用意

    # 前進代入
    for i in range(n):
        y[i, :] = Q[i, :] - np.dot(L[i, :i], y[:i, :])

    x = np.zeros((n, m))  # 解ベクトル x を用意

    # 後退代入
    for i in range(n - 1, -1, -1):
        if np.abs(U[i, i]) < 1e-8:  # 0除算防止
            print(f"Warning: U[{i}, {i}] is nearly zero. Adding small value.")
            U[i, i] = 1e-8
        x[i, :] = (y[i, :] - np.dot(U[i, i+1:], x[i+1:, :])) / U[i, i]
    return x

def leastSquaresMethodLU(P: np.array, Q: np.array) -> np.array:
    # 最小二乗法で解を求める
    affine_matrix = eq_solve(P.T @ P, P.T) @ Q
    return affine_matrix

def leastSquaresMethod(P: np.array, Q: np.array) -> np.array:
    # 最小二乗法で解を求める
    affine_matrix = np.linalg.solve(P.T @ P, P.T) @ Q
    return affine_matrix

In [4]:
# データベースに接続
conn = sqlite3.connect("world.db")
c = conn.cursor()

# CREATE TABLE world (x INTEGER, y INTEGER, z INTEGER, hash TEXT, matrix_1_x REAL, matrix_1_y REAL, matrix_1_z REAL, matrix_2_x REAL, matrix_2_y REAL, matrix_2_z REAL)
# ワールドの座標系1のアフィン変換行列を取得
c.execute("SELECT * FROM world WHERE hash = ?", (hash1,))
rows1 = c.fetchall()
rows1 = [list(row) for row in rows1 ]

# ワールドの座標系2のアフィン変換行列を取得
c.execute("SELECT * FROM world WHERE hash = ?", (hash2,))
rows2 = c.fetchall()
rows2 = [list(row) for row in rows2]

# ワールドの座標系3のアフィン変換行列を取得
c.execute("SELECT * FROM world WHERE hash = ?", (hash3,))
rows3 = c.fetchall()
rows3 = [list(row) for row in rows3]

c.close()

# ワールドの座標系1と座標系2のアフィン変換行列から座標系1と座標系3のアフィン変換行列を求める
matrix_affine1 = np.array([
    [[1, 0, 0, rows1[0][4]],[0, 1, 0, rows1[0][5]],[0, 0, 1, rows1[0][6]],[0, 0, 0, 1]],
    [[1, 0, 0, rows2[0][4]],[0, 1, 0, rows2[0][5]],[0, 0, 1, rows2[0][6]],[0, 0, 0, 1]],
    [[1, 0, 0, rows3[0][4]],[0, 1, 0, rows3[0][5]],[0, 0, 1, rows3[0][6]],[0, 0, 0, 1]],
])

matrix_affine2 = np.array([
    [[1, 0, 0, rows1[0][7]],[0, 1, 0, rows1[0][8]],[0, 0, 1, rows1[0][9]],[0, 0, 0, 1]],
    [[1, 0, 0, rows2[0][7]],[0, 1, 0, rows2[0][8]],[0, 0, 1, rows2[0][9]],[0, 0, 0, 1]],
    [[1, 0, 0, rows3[0][7]],[0, 1, 0, rows3[0][8]],[0, 0, 1, rows3[0][9]],[0, 0, 0, 1]],
])

affine = generateAffineMatrix3D(matrix_affine1, matrix_affine2)
print(affine)
affine_self = generateAffineMatrix3DSelf(matrix_affine1, matrix_affine2)
print(affine_self)
affine_self_LU = generateAffineMatrix3DSelfLU(matrix_affine1, matrix_affine2)
print(affine_self_LU)

affine_matrix
[[ 0.7387218  -0.34398496 -0.43045113]
 [ 0.2612782   1.34398496  0.43045113]
 [ 0.2612782   0.34398496  1.43045113]
 [-0.0093985   0.08834586  0.09962406]]
[[ 0.7387218   0.2612782   0.2612782  -0.0093985 ]
 [-0.34398496  1.34398496  0.34398496  0.08834586]
 [-0.43045113  0.43045113  1.43045113  0.09962406]
 [ 0.          0.          0.          1.        ]]
[[ 1.48265235 -0.48265235 -0.48265235 17.10100398]
 [ 0.49176939  0.50823061 -0.49176939 19.31069596]
 [ 2.36543623 -2.36543623 -1.36543623 64.4050332 ]
 [ 0.          0.          0.          1.        ]]
[[1.         0.         0.         0.        ]
 [0.90721649 1.         0.         0.        ]
 [2.93814433 1.23893805 1.         0.        ]
 [0.12371134 0.09734513 0.04347826 1.        ]]
[[1.94000000e+02 1.76000000e+02 5.70000000e+02 2.40000000e+01]
 [0.00000000e+00 2.32989691e+00 2.88659794e+00 2.26804124e-01]
 [0.00000000e+00 0.00000000e+00 4.68141593e+00 2.03539823e-01]
 [0.00000000e+00 0.00000000e+00 0.0000000

In [5]:
# hash = "70163de61cd8d40eb0d70587c380b89c16e6620c2cd3862d5fc9a62bd6134e9c"
hash = "223"
# hash = "87cb9455225e82801c55d7f005a4fda7f7080f2a3bfa1f6ee9cb91e91a6d45ca"

In [6]:
conn = sqlite3.connect("world.db")
c = conn.cursor()

c.execute("SELECT * FROM world WHERE hash = ?", (hash,))
rows1 = c.fetchall()
rows1 = [list(row) for row in rows1 ]
print(rows1)

c.close()

matrix = np.array([
    rows1[0][4],
    rows1[0][5],
    rows1[0][6],
    1
])

print(matrix)
print(affine)

affine_times_matrix = np.dot(affine, matrix)

print(affine_times_matrix)
print(rows1[0][7],rows1[0][8],rows1[0][9])

[[2, 2, 3, '223', 6.0, 7.0, 23.0, 12.0, 15.0, 33.0]]
[ 6.  7. 23.  1.]
[[ 0.7387218   0.2612782   0.2612782  -0.0093985 ]
 [-0.34398496  1.34398496  0.34398496  0.08834586]
 [-0.43045113  0.43045113  1.43045113  0.09962406]
 [ 0.          0.          0.          1.        ]]
[12.2612782  15.34398496 33.43045113  1.        ]
12.0 15.0 33.0


In [7]:
conn = sqlite3.connect("world.db")
c = conn.cursor()

c.execute("SELECT * FROM world WHERE hash = ?", (hash,))
rows1 = c.fetchall()
rows1 = [list(row) for row in rows1 ]
print(rows1)

c.close()

matrix = np.array([
    rows1[0][4],
    rows1[0][5],
    rows1[0][6],
    1
])

print(matrix)
print(affine_self)

affine_self_times_matrix = np.dot(affine_self, matrix)

print(affine_self_times_matrix)
print(rows1[0][7],rows1[0][8],rows1[0][9])

[[2, 2, 3, '223', 6.0, 7.0, 23.0, 12.0, 15.0, 33.0]]
[ 6.  7. 23.  1.]
[[ 1.48265235 -0.48265235 -0.48265235 17.10100398]
 [ 0.49176939  0.50823061 -0.49176939 19.31069596]
 [ 2.36543623 -2.36543623 -1.36543623 64.4050332 ]
 [ 0.          0.          0.          1.        ]]
[11.51734765 14.50823061 30.63456377  1.        ]
12.0 15.0 33.0


In [8]:
conn = sqlite3.connect("world.db")
c = conn.cursor()

c.execute("SELECT * FROM world WHERE hash = ?", (hash,))
rows1 = c.fetchall()
rows1 = [list(row) for row in rows1 ]
print(rows1)

c.close()

matrix = np.array([
    rows1[0][4],
    rows1[0][5],
    rows1[0][6],
    1
])

print(matrix)
print(affine_self_LU)

affine_self_LU_times_matrix = np.dot(affine_self_LU, matrix)

print(affine_self_LU_times_matrix)
print(rows1[0][7],rows1[0][8],rows1[0][9])

[[2, 2, 3, '223', 6.0, 7.0, 23.0, 12.0, 15.0, 33.0]]
[ 6.  7. 23.  1.]
[[ 7.39130446e-01  2.60869554e-01  2.60869554e-01  2.66453526e-07]
 [-3.47826077e-01  1.34782608e+00  3.47826077e-01  2.22044605e-07]
 [-4.34782573e-01  4.34782573e-01  1.43478257e+00  8.21565038e-07]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[12.26086955 15.34782608 33.43478257  1.        ]
12.0 15.0 33.0


In [9]:
conn = sqlite3.connect("world.db")
c = conn.cursor()

c.execute("SELECT * FROM world WHERE hash = ?", (hash,))
rows1 = c.fetchall()
rows1 = [list(row) for row in rows1 ]
print(rows1)

c.close()

matrix = np.array([
    rows1[0][4],
    rows1[0][5],
    rows1[0][6],
    1
])

swiftFromAffineMatrix = np.array([[-26.546425, -909.00305, 990.8738, 187.4226], [-28.166004, -965.0033, 1051.8181, 200.4558], [-26.921429, -922.0031, 1005.01544, 190.53925], [0.0, 0.0, 0.0, 1.0]])

print(matrix)
print(swiftFromAffineMatrix)

swiftFromAffineMatrix_times_matrix = np.dot(swiftFromAffineMatrix, matrix)

print(swiftFromAffineMatrix_times_matrix)
print(rows1[0][7],rows1[0][8],rows1[0][9])


[[2, 2, 3, '223', 6.0, 7.0, 23.0, 12.0, 15.0, 33.0]]
[ 6.  7. 23.  1.]
[[-2.65464250e+01 -9.09003050e+02  9.90873800e+02  1.87422600e+02]
 [-2.81660040e+01 -9.65003300e+02  1.05181810e+03  2.00455800e+02]
 [-2.69214290e+01 -9.22003100e+02  1.00501544e+03  1.90539250e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[1.64552201e+04 1.74682530e+04 1.66903441e+04 1.00000000e+00]
12.0 15.0 33.0


In [10]:
print(hash1, hash2, hash3)
print(hash)
# print(affine)
print(affine_times_matrix)
# print(affine_self)
print(affine_self_times_matrix)
# print(affine_self_LU)
print(affine_self_LU_times_matrix)
# print(swiftFromAffineMatrix)
print(swiftFromAffineMatrix_times_matrix)
print(rows1[0][7],rows1[0][8],rows1[0][9])

323 525 433
223
[12.2612782  15.34398496 33.43045113  1.        ]
[11.51734765 14.50823061 30.63456377  1.        ]
[12.26086955 15.34782608 33.43478257  1.        ]
[1.64552201e+04 1.74682530e+04 1.66903441e+04 1.00000000e+00]
12.0 15.0 33.0


| |  	   1   |
|---|---|
|1 |	25.00000008 |
|2 |	23.99999987 |
|3 |	42.99999999 |
|4 |	1 |

---

|	|   1    |
|---|---|
|1|	24.99999982 |
|2|	24.00000002 |
|3|	42.99999995 |
|4|	1 |
 
